# 물체인식 및 분류
테스트 - 학습 - Validation

## 1 OverView
간단하게 살펴보기

In [1]:
# setup-only-ignore
import tensorflow as tf
import numpy as np

tf.nn.conv2d    # 컨볼루션 레이어
tf.nn.relu      # relu 함수
tf.nn.max_pool  # Pooling 레이어 
tf.matmul       # Fully Connected 레이어

<function tensorflow.python.ops.math_ops.matmul>

In [2]:
# setup-only-ignore
sess = tf.InteractiveSession()

image_batch = tf.constant([
        [  # First Image : 높이 2pixel, 폭 3pixel
            [[0, 255, 0], [0, 255, 0], [0, 255, 0]],
            [[0, 255, 0], [0, 255, 0], [0, 255, 0]]],
        [  # Second Image : 높이 2pixel, 폭 3pixel
            [[0, 0, 255], [0, 0, 255], [0, 0, 255]],
            [[0, 0, 255], [0, 0, 255], [0, 0, 255]]] ])
image_batch.get_shape()

TensorShape([Dimension(2), Dimension(2), Dimension(3), Dimension(3)])

In [3]:
# First Image, 첫 픽셀, 색상값 출력 (list indexing)
# [ red , green , Yellow ]
sess.run(image_batch)[0][0][0]

array([  0, 255,   0], dtype=int32)

## 2 Convolution Nural Network
tf.nn.conv2d() 실험시 가장 많이 사용되는 layer

입력 tensor 의 차원을 줄이는 능력

### Input & Kernal

In [ ]:
sess.close()
# setup-only-ignore
sess = tf.InteractiveSession()

In [ ]:
# 입력 tensor
input_batch = tf.constant([
        [  # First Input
            [[0.0], [1.0]],
            [[2.0], [3.0]]],
        [  # Second Input
            [[2.0], [4.0]],
            [[6.0], [8.0]]] ])

# 조작 Kernel ( weight, filter, convolution matrix, mask 으로도 불린다 )
# 원본값 그대로 출력 channel, 2배 증폭 channel
kernel = tf.constant([
        [[[1.0, 2.0]]] ])

In [7]:
# 커널의 연관된 값으로 곱한 후, Feature map 의 연관된 레이어에 더해진다.
conv2d = tf.nn.conv2d(input_batch, kernel, strides=[1, 1, 1, 1], padding='SAME')

#lower_right_image_pixel
sess.run(input_batch)[0][1][1]

array([ 3.], dtype=float32)

In [8]:
#lower_right_kernel_pixel
sess.run(conv2d)[0][1][1]

array([ 3.,  6.], dtype=float32)

### Stride 
kernel 적용식 큰 걸음 걷기

In [21]:
sess.close()
import tensorflow as tf
sess = tf.InteractiveSession()

input_batch = tf.constant([
        [  # First Input (6x6x1)
            [[0.0], [1.0], [2.0], [3.0], [4.0], [5.0]],
            [[0.1], [1.1], [2.1], [3.1], [4.1], [5.1]],
            [[0.2], [1.2], [2.2], [3.2], [4.2], [5.2]],
            [[0.3], [1.3], [2.3], [3.3], [4.3], [5.3]],
            [[0.4], [1.4], [2.4], [3.4], [4.4], [5.4]],
            [[0.5], [1.5], [2.5], [3.5], [4.5], [5.5]], ]])

kernel = tf.constant([  # Kernel (3x3x1)
        [[[0.0]], [[0.5]], [[0.0]]],
        [[[0.0]], [[1.0]], [[0.0]]],
        [[[0.0]], [[0.5]], [[0.0]]]  ])

# stride = [입력 tensor , 3 x 3 stride, 1 layer ]
conv2d = tf.nn.conv2d(input_batch, kernel, strides=[1, 3, 3, 1], padding='SAME')
sess.run(conv2d)

array([[[[ 2.20000005],
         [ 8.19999981]],

        [[ 2.79999995],
         [ 8.80000019]]]], dtype=float32)

### Padding 
커널이 이미지에 겹쳐질 때 모자란 부분을 채우는 연산

same, valid

In [ ]:
sess.close()
# setup-only-ignore

In [26]:
import matplotlib as mil
#mil.use('svg')
mil.use("nbagg")
from matplotlib import pyplot
fig = pyplot.gcf()
fig.set_size_inches(4, 4)

In [27]:
image_filename = "./data/n02113023_219.jpg"
filename_queue = tf.train.string_input_producer(
    tf.train.match_filenames_once(image_filename))

In [28]:
image_reader = tf.WholeFileReader()
_, image_file = image_reader.read(filename_queue)
image = tf.image.decode_jpeg(image_file)

In [31]:
sess.run(tf.global_variables_initializer().run())

ValueError: Cannot execute operation using `run()`: No default session is registered. Use `with sess.as_default():` or pass an explicit session to `run(session=sess)`

In [30]:
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

image_batch = tf.image.convert_image_dtype(tf.expand_dims(image, 0), tf.float32, saturate=False)

ValueError: Cannot start queue runners: No default session is registered. Use `with sess.as_default()` or pass an explicit session to tf.start_queue_runners(sess=sess)

In [23]:
kernel = tf.constant([
        [
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]
        ],
        [
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ 8., 0., 0.], [ 0., 8., 0.], [ 0., 0., 8.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]
        ],
        [
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]
        ]
    ])


conv2d = tf.nn.conv2d(image_batch, kernel, [1, 1, 1, 1], padding="SAME")
activation_map = sess.run(tf.minimum(tf.nn.relu(conv2d), 255))

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.FailedPreconditionError'>, Attempting to use uninitialized value matching_filenames
	 [[Node: matching_filenames/read = Identity[T=DT_STRING, _class=["loc:@matching_filenames"], _device="/job:localhost/replica:0/task:0/cpu:0"](matching_filenames)]]
	 [[Node: input_producer/Size/_3 = _HostRecv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_5_input_producer/Size", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]


OutOfRangeError: FIFOQueue '_0_input_producer' is closed and has insufficient elements (requested 1, current size 0)
	 [[Node: ReaderReadV2 = ReaderReadV2[_device="/job:localhost/replica:0/task:0/cpu:0"](WholeFileReaderV2, input_producer)]]
	 [[Node: DecodeJpeg/_7 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/gpu:0", send_device="/job:localhost/replica:0/task:0/cpu:0", send_device_incarnation=1, tensor_name="edge_6_DecodeJpeg", tensor_type=DT_UINT8, _device="/job:localhost/replica:0/task:0/gpu:0"]()]]

Caused by op 'ReaderReadV2', defined at:
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/markbaum/Python/python36/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/markbaum/Python/python36/lib/python3.6/site-packages/traitlets/config/applicat

In [ ]:
# setup-only-ignore
fig = pyplot.gcf()
pyplot.imshow(activation_map[0], interpolation='nearest')
#pyplot.show()
fig.set_size_inches(4, 4)
#fig.savefig("./images/chapter-05-object-recognition-and-classification/convolution/example-edge-detection.png")

In [ ]:
kernel = tf.constant([
        [
            [[ 0., 0., 0.], [ 0., 0., 0.], [ 0., 0., 0.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ 0., 0., 0.], [ 0., 0., 0.], [ 0., 0., 0.]]
        ],
        [
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ 5., 0., 0.], [ 0., 5., 0.], [ 0., 0., 5.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]]
        ],
        [
            [[ 0., 0., 0.], [ 0., 0., 0.], [ 0., 0., 0.]],
            [[ -1., 0., 0.], [ 0., -1., 0.], [ 0., 0., -1.]],
            [[ 0, 0., 0.], [ 0., 0., 0.], [ 0., 0., 0.]]
        ]
    ])


conv2d = tf.nn.conv2d(image_batch, kernel, [1, 1, 1, 1], padding="SAME")
activation_map = sess.run(tf.minimum(tf.nn.relu(conv2d), 255))

In [ ]:
# setup-only-ignore
fig = pyplot.gcf()
pyplot.imshow(activation_map[0], interpolation='nearest')
#pyplot.show()
fig.set_size_inches(4, 4)
#fig.savefig("./images/chapter-05-object-recognition-and-classification/convolution/example-sharpen.png")

In [ ]:
# setup-only-ignore
filename_queue.close(cancel_pending_enqueues=True)
coord.request_stop()
coord.join(threads)

In [ ]:
from sklearn.neural_network import MLPClassifier
X = [[0., 0.], [1., 1.]]
y = [0, 1]
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(5, 2), random_state=1)
clf.fit(X, y) 

In [ ]:
clf.predict([[2., 2.], [-1., -2.]])

In [ ]:
[coef.shape for coef in clf.coefs_]

In [ ]:
clf.predict_proba([[2., 2.], [1., 2.]])  

In [ ]:
## CPU vs GPU processing
from __future__ import print_function
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
import time

In [ ]:
def get_times(maximum_time):
    device_times = {"/gpu:0":[],"/cpu:0":[]}
    matrix_sizes = range(500,50000,50)

    for size in matrix_sizes:
        for device_name in device_times.keys():
            print("####### Calculating on the " + device_name + " #######")
            shape = (size,size)
            data_type = tf.float16
            with tf.device(device_name):
                r1 = tf.random_uniform(shape=shape, minval=0, maxval=1, dtype=data_type)
                r2 = tf.random_uniform(shape=shape, minval=0, maxval=1, dtype=data_type)
                dot_operation = tf.matmul(r2, r1)

            with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as session:
                start_time = time.time()
                result = session.run(dot_operation)
                time_taken = time.time() - start_time
                print(result)
                device_times[device_name].append(time_taken)

            print(device_times)
            if time_taken > maximum_time:
                return device_times, matrix_sizes

In [ ]:
device_times, matrix_sizes = get_times(1.5)
gpu_times = device_times["/gpu:0"]
cpu_times = device_times["/cpu:0"]

In [ ]:
plt.plot(matrix_sizes[:len(gpu_times)], gpu_times, 'o-')
plt.plot(matrix_sizes[:len(cpu_times)], cpu_times, 'o-')
plt.ylabel('Time')
plt.xlabel('Matrix size')
plt.show()